In [ ]:
# !pip install sklearn, pennylane, matplotlib

# Comparing kernel methods and variational quantum circuits on a real dataset

In [1]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt

/Users/gbltostibalduc/miniforge3/envs/plTutorialsEnv/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Declare a random number generator for reproducibility

In [2]:
rng = np.random.default_rng(42)

## Loading and preparing the Iris dataset

In [3]:
X, y = load_iris(return_X_y=True)

# pick only the first two classes (corresponding to the first 100 samples)
X = X[:100]
y = y[:100]

# scaling the data for better training and avoiding problems due to periodic encoding
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# move labels from [0, 1] to [-1, 1] range
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)


## Angle encoding

Exact amplitude encoding may require up to an exponential amount of operations and thus it does not work for NISQ. We can do another type of encoding, i.e. *angle encoding*, for which the features are encoded in as many qubits and each of them is used as a rotation angle of a gate.

In [4]:
n_qubits = len(X_train[0])

Let's create a function that encodes each of the features as a $R_Z$ rotation for each qubit

In [5]:
def angle_encoding_rz(x):
    """Encodes features as Z-rotations

    Args:
        x (np.ndarray): data point where the entries are the features to be encoded
    """
    # ================
    # YOUR CODE BELOW
    # ================

## Computing the kernel

In quantum kernel methods, the quantum computer is used exclusively to compute the kernel matrix. 

Remember that the kernel is given by the inner product of two samples $x_1$ and $x_2$. More correctly, it is the inner product between their mappings in the higher-dimensional feature space, thus $\phi(x_1)$ and $\phi(x_2)$. For us, this higher-dimensional mapping is given by our $R_Z$-encoding, which leads to the states $|\phi(x_1)\rangle$ and $|\phi(x_2)\rangle$. Therefore, we want to compute $|\langle \phi(x_2) | \phi(x_1) \rangle|^2$ (the square matters because the kernel is a measure of a distance).

It is easy to verify that the circuit used to compute the kernel is

<img src="kernel.jpg" alt="kernel" width="200"/>

where $\Phi(x)$ computes the feature map and the final measuerement is on the all-zeros projector $|0\dots 0\rangle\langle 0\dots 0|$ (equivalent to computing the probability of measuring the all-zeros state).